In [1]:
import subprocess
import json
import os
import pickle
import traceback
import shutil
import socket

import numpy as np
import torch


process = subprocess.Popen('docker container start mobilitydb', shell=True)

In [2]:
hostname = socket.gethostname()
test = hostname.split("-")[-1]
print("test", test)

test dev


In [3]:
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            # Jupyter notebook or qtconsole
            return True
        elif shell == 'TerminalInteractiveShell':
            # Terminal running IPython
            return False
        else:
            # Other type (?)
            return False
    except NameError:
        # Probably standard Python interpreter
        return False


if is_notebook():
    %cd ..
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

/home/chanwutk/Documents/apperception
mobilitydb


In [4]:
process.wait()

0

In [5]:
from optimized_ingestion.camera_config import camera_config
from optimized_ingestion.payload import Payload
from optimized_ingestion.pipeline import Pipeline
from optimized_ingestion.video import Video
from optimized_ingestion.metadata_json_encoder import MetadataJSONEncoder

In [6]:
# Stages
from optimized_ingestion.stages.in_view import InView

from optimized_ingestion.stages.decode_frame.decode_frame import DecodeFrame

from optimized_ingestion.stages.detection_2d.detection_2d import Detection2D
from optimized_ingestion.stages.detection_2d.yolo_detection import YoloDetection
from optimized_ingestion.stages.detection_2d.object_type_filter import ObjectTypeFilter
from optimized_ingestion.stages.detection_2d.ground_truth import GroundTruthDetection

In [7]:
from optimized_ingestion.stages.detection_3d.from_detection_2d_and_road import FromDetection2DAndRoad
from optimized_ingestion.stages.detection_3d.from_detection_2d_and_depth import FromDetection2DAndDepth

from optimized_ingestion.stages.depth_estimation import DepthEstimation

from optimized_ingestion.stages.detection_estimation import DetectionEstimation
from optimized_ingestion.stages.detection_estimation.segment_mapping import RoadPolygonInfo

In [8]:
from optimized_ingestion.stages.tracking_2d.strongsort import StrongSORT

In [9]:
from optimized_ingestion.stages.tracking_2d.tracking_2d import Tracking2D, Tracking2DResult

In [10]:
from optimized_ingestion.stages.tracking_3d.from_tracking_2d_and_road import FromTracking2DAndRoad
from optimized_ingestion.stages.tracking_3d.from_tracking_2d_and_depth import FromTracking2DAndDepth
from optimized_ingestion.stages.tracking_3d.tracking_3d import Tracking3DResult, Tracking3D

from optimized_ingestion.stages.segment_trajectory import SegmentTrajectory
from optimized_ingestion.stages.segment_trajectory.construct_segment_trajectory import SegmentPoint
from optimized_ingestion.stages.segment_trajectory.from_tracking_3d import FromTracking3D

In [11]:
from optimized_ingestion.cache import disable_cache
disable_cache()

In [12]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
print(NUSCENES_PROCESSED_DATA in os.environ)
print(os.environ['NUSCENES_PROCESSED_DATA'])

True
/home/chanwutk/data/processed/full-dataset/trainval


In [13]:
DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
with open(os.path.join(DATA_DIR, "videos", "frames.pkl"), "rb") as f:
    videos = pickle.load(f)

In [14]:
with open(os.path.join(DATA_DIR, 'cities.pkl'), 'rb') as f:
    cities = pickle.load(f)

In [15]:
BENCHMARK_DIR = "./outputs/ablation-performance-run"


def bm_dir(*args: "str"):
    return os.path.join(BENCHMARK_DIR, *args)

In [16]:
def run_benchmark(pipeline, filename, run=0, ignore_error=False):
    metadata_strongsort = {}
    metadata_3d = {}
    metadata_segment = {}
    metadata_d2d = {}
    failed_videos = []

    all_metadata = {
        'sort': metadata_strongsort,
        'segment-trajectory': metadata_segment,
        'detection-2d': metadata_d2d,
    }

    names = cities['boston-seaport'][:100]
    filtered_videos = [(n, v) for n, v in videos.items() if n[6:10] in names]

    for pre in all_metadata.keys():
        p = os.path.join(BENCHMARK_DIR, f"{pre}--{filename}_{run}")
        if os.path.exists(p):
            shutil.rmtree(p)
        os.makedirs(p)

    for i, (name, video) in tqdm(enumerate(filtered_videos), total=len(filtered_videos)):
        try:
            video_filename = video['filename']
            if not video_filename.startswith('boston') or 'FRONT' not in name:
                continue

            frames = Video(
                os.path.join(DATA_DIR, "videos", video["filename"]),
                [camera_config(*f, 0) for f in video["frames"]],
            )

            output = pipeline.run(Payload(frames))

            metadata_strongsort[name] = output[StrongSORT]
            metadata_3d[name] = output[Tracking3D]
            metadata_segment[name] = output[SegmentTrajectory]
            metadata_d2d[name] = [
                (d2ds, dids)
                for d2ds, clss, dids
                in output[Detection2D]
            ]

            for pre, metadata in all_metadata.items():
                p = bm_dir(f"{pre}--{filename}_{run}", f"{name}.json")
                with open(p, "w") as f:
                    json.dump(metadata[name], f, cls=MetadataJSONEncoder, indent=1)
        except Exception as e:
            if ignore_error:
                message = str(traceback.format_exc())
                failed_videos.append((name, message))
                print(video_filename)
                print(e)
                print(message)
                print("------------------------------------------------------------------------------------")
                print()
                print()
            else:
                raise e

        if i % 20 == 0:
            with open(bm_dir(f"failed_videos--{filename}_{run}.json"), "w") as f:
                json.dump(failed_videos, f, indent=1)

            with open(bm_dir(f"perf--{filename}_{run}.json"), "w") as f:
                performance = [
                    {
                        "stage": stage.classname(),
                        "benchmark": stage.benchmark,
                        **(
                            {'explains': stage.explains}
                            if hasattr(stage, 'explains')
                            else {}
                        )
                    }
                    for stage
                    in pipeline.stages
                ]
                json.dump(performance, f, indent=1)

In [17]:
def create_pipeline(
    predicate,
    in_view=True,
    object_filter=True,
    groundtruth_detection=False,
    geo_depth=True,
    detection_estimation=True,
    strongsort=False,
):
    pipeline = Pipeline()

    # In-View Filter
    if in_view:
        # TODO: view angle and road type should depends on the predicate
        pipeline.add_filter(InView(50, 'intersection'))

    # Decode
    pipeline.add_filter(DecodeFrame())

    # 2D Detection
    if groundtruth_detection:
        with open(os.path.join(DATA_DIR, 'annotation_partitioned.pkl'), 'rb') as f:
            df_annotations = pickle.load(f)
        pipeline.add_filter(GroundTruthDetection(df_annotations))
    else:
        pipeline.add_filter(YoloDetection())

    # Object Filter
    if object_filter:
        # TODO: filter objects based on predicate
        pipeline.add_filter(ObjectTypeFilter(['car']))

    # 3D Detection
    if geo_depth:
        pipeline.add_filter(FromDetection2DAndRoad())
    else:
        pipeline.add_filter(DepthEstimation())
        pipeline.add_filter(FromDetection2DAndDepth())

    # Detection Estimation
    if detection_estimation:
        pipeline.add_filter(DetectionEstimation())

    # Tracking
    pipeline.add_filter(StrongSORT(cache=strongsort))

    # Tracking 3D
    if geo_depth:
        pipeline.add_filter(FromTracking2DAndRoad())
    else:
        pipeline.add_filter(FromTracking2DAndDepth())

    # Segment Trajectory
    pipeline.add_filter(FromTracking3D())

    return pipeline

In [18]:
predicate = None

p_noOpt = lambda predicate: create_pipeline(
    predicate,
    in_view=False,
    object_filter=False,
    geo_depth=False,
    detection_estimation=False
)

p_inview = lambda predicate: create_pipeline(
    predicate,
    in_view=True,
    object_filter=False,
    geo_depth=False,
    detection_estimation=False
)

p_objectFilter = lambda predicate: create_pipeline(
    predicate,
    in_view=False,
    object_filter=True,
    geo_depth=False,
    detection_estimation=False
)

p_geo = lambda predicate: create_pipeline(
    predicate,
    in_view=False,
    object_filter=False,
    geo_depth=True,
    detection_estimation=False
)

p_de = lambda predicate: create_pipeline(
    predicate,
    in_view=False,
    object_filter=False,
    geo_depth=False,
    detection_estimation=True
)

p_opt = lambda predicate: create_pipeline(
    predicate,
    in_view=True,
    object_filter=True,
    geo_depth=True,
    detection_estimation=False
)

p_optDe = lambda predicate: create_pipeline(
    predicate,
    in_view=True,
    object_filter=True,
    geo_depth=True,
    detection_estimation=True
)

p_gtOpt = lambda predicate: create_pipeline(
    predicate,
    in_view=True,
    object_filter=True,
    groundtruth_detection=True,
    geo_depth=True,
    detection_estimation=False
)

p_gtOptDe = lambda predicate: create_pipeline(
    predicate,
    in_view=True,
    object_filter=True,
    groundtruth_detection=True,
    geo_depth=True,
    detection_estimation=True
)

pipelines = {
    "noopt": p_noOpt,
    "inview": p_inview,
    "objectfilter": p_objectFilter,
    "geo": p_geo,
    "de": p_de,
    "opt": p_opt,
    "optde": p_optDe,
    "gtopt": p_gtOpt,
    "gtoptde": p_gtOptDe
}

In [19]:
if test == 'dev':
    test = 'opt'
for i in range(3):
    run_benchmark(pipelines[test](None), test, run=i, ignore_error=True)

  0%|          | 0/600 [00:00<?, ?it/s]

boston-seaport-scene-0068-CAM_FRONT.mp4
module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
Traceback (most recent call last):
  File "/tmp/ipykernel_11517/1164415758.py", line 34, in run_benchmark
    output = pipeline.run(Payload(frames))
  File "/home/chanwutk/Documents/apperception/optimized_ingestion/pipeline.py", line 22, in run
    payload = payload.filter(stage)
  File "/home/chanwutk/Documents/apperception/optimized_ingestion/payload.py", line 36, in filter
    keep, metadata = filter.run(self)
  File "/home/chanwutk/Documents/apperception/optimized_ingestion/stag

KeyboardInterrupt: 

In [ ]:
# subprocess.Popen('shutdown -h now', shell=True)

In [ ]:
test = 'deopt'
for i in range(3):
    run_benchmark(pipelines[test](None), test, run=i, ignore_error=True)